# Kernel Logistic Regression

* **Version 1-3:** Nystroem Kernel Approximation + Logistic Regression
* **Version 4:** Kernel Ridge Regression + Platt Scaling
* **Version 5:** Kernel Ridge Regression + Platt Scaling + Tuned Hyperparameter C's
* **Version 6:** Kernel Ridge Regression + Inter-Target Platt Scaling (CPU) The training time is too long so I cancelled it.
* **Version 7:** Kernel Ridge Regression + Inter-Target Platt Scaling (GPU)
* **Version 8:** Kernel Ridge Regression + Platt Scaling (CPU) + Remove Control Group
* **Version 9-13:** GroupCV + Quantile Transformation + Label Smoothing
* **Version 14:** Add PCA and VarianceThreshold Feature Selection

In this example, I play with the kernel logistic regression method. Scikit-Learn does not have kernel logistic regression. Instead, I use kernel ridge regression and platt scaling. According to the [Kernel Ridge Regression][1] document on Scikit-Learn, It should perform as well as SVR.

P.S. The inter-target Platt Scaling means I consider target relationships during Platt Scaling.

[1]: https://scikit-learn.org/stable/modules/generated/sklearn.kernel_ridge.KernelRidge.html#sklearn.kernel_ridge.KernelRidge

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

# Thanks to Chris's RAPIDS dataset, it only takes around 1 min to install offline
# !cp ../input/rapids/rapids.0.15.0 /opt/conda/envs/rapids.tar.gz
# !cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
# sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
# sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
# sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
# !cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
import os
import gc
import datetime
import numpy as np
import pandas as pd
from numba import njit
from abc import abstractmethod, ABCMeta
from sklearn.kernel_approximation import Nystroem
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
# from cuml import LogisticRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm
from time import time

In [ ]:
def create_folds(num_starts, num_splits):
    
    folds = []
    
    # LOAD FILES
    train_feats = pd.read_csv('../input/lish-moa/train_features.csv')
    scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
    drug = pd.read_csv('/kaggle/input/lish-moa/train_drug.csv')
    scored = scored.loc[train_feats['cp_type'] == 'trt_cp', :]
    drug = drug.loc[train_feats['cp_type'] == 'trt_cp', :]
    targets = scored.columns[1:]
    scored = scored.merge(drug, on = 'sig_id', how = 'left') 

    # LOCATE DRUGS
    vc = scored.drug_id.value_counts()
    vc1 = vc.loc[vc <= 18].index.sort_values()
    vc2 = vc.loc[vc > 18].index.sort_values()
    
    for seed in range(num_starts):

        # STRATIFY DRUGS 18X OR LESS
        dct1 = {}; dct2 = {}
        skf = MultilabelStratifiedKFold(n_splits = num_splits, shuffle = True, random_state = seed)
        tmp = scored.groupby('drug_id')[targets].mean().loc[vc1]
        for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[targets])):
            dd = {k:fold for k in tmp.index[idxV].values}
            dct1.update(dd)

        # STRATIFY DRUGS MORE THAN 18X
        skf = MultilabelStratifiedKFold(n_splits = num_splits, shuffle = True, random_state = seed)
        tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop = True)
        for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[targets])):
            dd = {k:fold for k in tmp.sig_id[idxV].values}
            dct2.update(dd)

        # ASSIGN FOLDS
        scored['fold'] = scored.drug_id.map(dct1)
        scored.loc[scored.fold.isna(),'fold'] =\
            scored.loc[scored.fold.isna(),'sig_id'].map(dct2)
        scored.fold = scored.fold.astype('int8')
        folds.append(scored.fold.values)
        
        del scored['fold']
        
    return np.stack(folds)

# Data Preparation

In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')

ss_krr = pd.read_csv('../input/lish-moa/sample_submission.csv')
ss_lr = ss_krr.copy()

cols = [c for c in ss_krr.columns.values if c != 'sig_id']
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [ ]:
def preprocess(df):
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 0, 48: 0.5, 72: 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    del df['sig_id']
    return df

def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    return - np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip))

train = preprocess(train_features)
test = preprocess(test_features)

del train_targets['sig_id']
del train_targets_nonscored['sig_id']

In [ ]:
from sklearn.preprocessing import QuantileTransformer

qt = QuantileTransformer(n_quantiles = 100, output_distribution = 'normal', random_state = 42)
qt.fit(pd.concat([pd.DataFrame(train[GENES+CELLS]), pd.DataFrame(test[GENES+CELLS])]))
train[GENES+CELLS] = qt.transform(train[GENES+CELLS])
test[GENES+CELLS] = qt.transform(test[GENES+CELLS])

In [ ]:
from sklearn.decomposition import PCA

# GENES
n_comp_genes = 600

data = pd.concat([pd.DataFrame(train[GENES]), pd.DataFrame(test[GENES])])
pca_genes = PCA(n_components=n_comp_genes, random_state = 42)
data2 = pca_genes.fit_transform(data[GENES])
train2 = data2[:train.shape[0]]; test2 = data2[-test.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp_genes)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp_genes)])

train = pd.concat((train, train2), axis=1)
test = pd.concat((test, test2), axis=1)

#CELLS
n_comp_cells = 50

data = pd.concat([pd.DataFrame(train[CELLS]), pd.DataFrame(test[CELLS])])
pca_cells = PCA(n_components=n_comp_cells, random_state = 42)
data2 = pca_cells.fit_transform(data[CELLS])
train2 = data2[:train.shape[0]]; test2 = data2[-test.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp_cells)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp_cells)])

train = pd.concat((train, train2), axis=1)
test = pd.concat((test, test2), axis=1)

In [ ]:
from sklearn.feature_selection import VarianceThreshold

var_thresh = VarianceThreshold(0.8)
data = train.append(test)
data_transformed = var_thresh.fit_transform(data.iloc[:, 3:])

train_transformed = data_transformed[ : train.shape[0]]
test_transformed = data_transformed[-test.shape[0] : ]

train = pd.DataFrame(train[['cp_type','cp_time','cp_dose']].values.reshape(-1, 3),\
            columns=['cp_type','cp_time','cp_dose'])

train = pd.concat([train, pd.DataFrame(train_transformed)], axis=1)

test = pd.DataFrame(test[['cp_type','cp_time','cp_dose']].values.reshape(-1, 3),\
            columns=['cp_type','cp_time','cp_dose'])

test = pd.concat([test, pd.DataFrame(test_transformed)], axis=1)

print(train.shape)
print(test.shape)

In [ ]:
train_targets = train_targets.loc[train['cp_type'] == 0].reset_index(drop = True)
train_targets_nonscored = train_targets_nonscored.loc[train['cp_type'] == 0].reset_index(drop = True)
train = train.loc[train['cp_type'] == 0].reset_index(drop = True)

print(train.shape)

In [ ]:
top_feats = np.arange(1, train.shape[1])
print(top_feats)

In [ ]:
train.head()

In [ ]:
N_SPLITS = 7
LBS = 0.0008
folds = create_folds(1, N_SPLITS)
print(folds)

# Kernel Logistic Regression

In [ ]:
res_krr = train_targets.copy()
ss_krr.loc[:, train_targets.columns] = 0
res_krr.loc[:, train_targets.columns] = 0

# for n, (tr, te) in enumerate(MultilabelStratifiedKFold(n_splits = N_SPLITS, random_state = 0, shuffle = True).split(train_targets, train_targets)):
for n, foldno in enumerate(set(folds[0])):
    start_time = time()
    tr = folds[0] != foldno
    te = folds[0] == foldno

    x_tr, x_val = train.values[tr][:, top_feats], train.values[te][:, top_feats]
    y_tr, y_val = train_targets.astype(float).values[tr], train_targets.astype(float).values[te]
    x_tt = test.values[:, top_feats]
    
    # Label Smoothing
    y_tr = y_tr * (1 - LBS) + 0.5 * LBS
    
    model = KernelRidge(alpha = 80, kernel = 'rbf')
    model.fit(x_tr, y_tr)

    ss_krr.loc[:, train_targets.columns] += model.predict(x_tt) / N_SPLITS
    fold_pred = model.predict(x_val)
    res_krr.loc[te, train_targets.columns] += fold_pred
    fold_score = log_loss_metric(train_targets.loc[te].values, fold_pred)
    print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] KRR: Fold {n}:', fold_score)

In [ ]:
print(f'Model OOF Metric: {log_loss_metric(train_targets.values, res_krr.values)}')

# Platt Scaling

Train a Logistic Regression model to calibrate the results of Kernel Ridge Regression.

In [ ]:
X_new = res_krr[cols].values
x_tt_new = ss_krr[cols].values

In [ ]:
res_lr = train_targets.copy()
ss_lr.loc[:, train_targets.columns] = 0
res_lr.loc[:, train_targets.columns] = 0

for tar in tqdm(range(train_targets.shape[1])):

#     start_time = time()
    targets = train_targets.values[:, tar]

    if targets.sum() >= N_SPLITS:

        skf = StratifiedKFold(n_splits = N_SPLITS, random_state = 0, shuffle = True)

        for n, (tr, te) in enumerate(skf.split(targets, targets)):

            x_tr, x_val = X_new[tr, tar].reshape(-1, 1), X_new[te, tar].reshape(-1, 1)
            y_tr, y_val = targets[tr], targets[te]

            model = LogisticRegression(penalty = 'none', max_iter = 1000)
            model.fit(x_tr, y_tr)
            ss_lr.loc[:, train_targets.columns[tar]] += model.predict_proba(x_tt_new[:, tar].reshape(-1, 1))[:, 1] / N_SPLITS
            res_lr.loc[te, train_targets.columns[tar]] += model.predict_proba(x_val)[:, 1]

    score = log_loss(train_targets.loc[:, train_targets.columns[tar]].values, res_lr.loc[:, train_targets.columns[tar]].values)
#     print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] LR Target {tar}:', score)

In [ ]:
print(f'LR OOF Metric: {log_loss_metric(train_targets.values, res_lr.values)}')

In [ ]:
np.save('klr_oof.npy', res_lr[cols].values)
np.save('klr_sub.npy', ss_lr[cols].values)

# Submit

In [ ]:
ss_lr.loc[test['cp_type'] == 1, train_targets.columns] = 0
ss_lr.to_csv('submission.csv', index = False)